In [1]:
# Parameters
msgs = "Ran from Airflow at 2021-11-09T17:27:40.842867+00:00!"


In [2]:
import requests
import time
import datetime
import pandas as pd
import io
import mysql.connector
from sqlalchemy import exc
import sqlalchemy
pd.options.display.max_rows = 150

In [3]:
def c_engine(host, username, pwd, db_name, attempts = 3) :
    try : 
        engine = sqlalchemy.create_engine('mysql+mysqlconnector://{user}:{password}@{host}/{database}'.format(
            user = username,
            password = pwd, 
            host = host,
            database = db_name))
        return engine
    
    except exc.SQLAlchemyError as error:
        if attempts>0:
            print("An error has occured ", error)
            time.sleep(30)
            c_engine(host, username, pwd, db_name, attempts-1)
    return False

def select_data(engine, table_name) : 
    try:
        sql_query = pd.read_sql_query(
            '''SELECT * FROM {table}'''.format(table = table_name), engine)
        dataframe = pd.DataFrame(sql_query)
        if dataframe.empty : 
            return False, None
        else : 
            return True, dataframe
    except exc.SQLAlchemyError as error:
        print("An error has occured", error)
        return False, None

In [4]:
#getting data
engine = c_engine("cvp19.mysql.database.azure.com", "u5487210@cvp19", "jRG2XXi4CwjHvAa", "my_db")
_, dataframe = select_data(engine, 'vaccins')

In [5]:
dataframe.dtypes

Date                      object
community                 object
Total doses delivered      int64
Doses administered         int64
% over delivered         float64
No. People vaccinated      int64
ccaa_iso                  object
dtype: object

In [6]:
#sorting dates and setting date column as index
dataframe["Date"] = pd.to_datetime(dataframe["Date"])
dataframe = dataframe.sort_values(["Date"]).set_index("Date")
dataframe

,community,Total doses delivered,Doses administered,% over delivered,No. People vaccinated,ccaa_iso
Date,,,,,,
2021-02-01,Andalucía,330980,313286,0.946541,90764,AN
2021-02-01,Navarra,28145,26779,0.951466,6320,NC
2021-02-01,Madrid,232775,199495,0.857029,24312,MD
2021-02-01,Murcia,60425,60731,1.005064,12580,MC
2021-02-01,Aragón,56505,56482,0.999593,11119,AR
...,...,...,...,...,...,...
2021-11-07,C. Valenciana,8088540,7831632,0.968238,4015726,VC
2021-11-07,Extremadura,1833705,1784224,0.973016,884894,EX
2021-11-07,Navarra,1046175,1017222,0.972325,522257,NC


In [7]:
def correct_dfs(dataframe, range_dates) : 
    
    full_dataframe = dataframe.loc["2021-02-01"]
    
    for date in range_dates : 

        short_dataframe = dataframe.loc[date.strftime("%Y-%m-%d")].groupby("community", as_index = False).max()
        nn_ccaa = [ccaa for ccaa in full_dataframe["community"].to_list() if ccaa not in short_dataframe["community"].to_list()]
        concat = pd.concat([short_dataframe, full_dataframe.loc[full_dataframe["community"].isin(nn_ccaa)]])
        concat.index = [date for i in range(len(concat.index.to_list()))]   
        
        dataframe = dataframe.loc[dataframe.index != date.strftime("%Y-%m-%d")]
        dataframe = dataframe.append(concat)
        full_dataframe = concat
        
    return dataframe


def unavailable_dates(df, dates):
    #sorting dates :
    dates_sorted = dates.sort()
    
    for date in dates : 
        print("We treating that date {date} ---".format(date = date))
        
        day_before = date - datetime.timedelta(days=1)
        
        sub_df = df.loc[day_before.strftime("%Y-%m-%d")].groupby("community", as_index = False).max()
        sub_df.index = [date for i in range(len(sub_df.index.to_list()))]
        df = df.append(sub_df)
        
        
    return df

In [8]:
test_range = [datetime.datetime(2021,1,15) + datetime.timedelta(days = i) for i in range(266)][18:]

In [9]:
missing_dates = [date for date in test_range if date not in dataframe.index.to_list()]
missing_dates

[]

In [10]:
corrected_dates = unavailable_dates(dataframe, missing_dates)

In [11]:
corrected_dates = corrected_dates.loc["2021-02-01":].sort_index()
corrected_dates

,community,Total doses delivered,Doses administered,% over delivered,No. People vaccinated,ccaa_iso
Date,,,,,,
2021-02-01,Andalucía,330980,313286,0.946541,90764,AN
2021-02-01,Navarra,28145,26779,0.951466,6320,NC
2021-02-01,Madrid,232775,199495,0.857029,24312,MD
2021-02-01,Murcia,60425,60731,1.005064,12580,MC
2021-02-01,Aragón,56505,56482,0.999593,11119,AR
...,...,...,...,...,...,...
2021-11-07,C. Valenciana,8088540,7831632,0.968238,4015726,VC
2021-11-07,Extremadura,1833705,1784224,0.973016,884894,EX
2021-11-07,Navarra,1046175,1017222,0.972325,522257,NC


In [12]:
final = correct_dfs(corrected_dates, test_range).sort_index()

In [13]:
final

,community,Total doses delivered,Doses administered,% over delivered,No. People vaccinated,ccaa_iso
2021-02-01,Andalucía,330980,313286,0.946541,90764,AN
2021-02-01,Extremadura,51975,49703,0.956287,10776,EX
2021-02-01,C. Valenciana,145830,147945,1.014503,44979,VC
2021-02-01,Asturias,59435,58177,0.978834,19503,AS
2021-02-01,Castilla y Leon,123895,126362,1.019912,35051,CL
...,...,...,...,...,...,...
2021-11-07,País Vasco,3719130,3455869,0.929214,1784368,PV
2021-11-07,C. Valenciana,8088540,7831632,0.968238,4015726,VC
2021-11-07,Extremadura,1833705,1784224,0.973016,884894,EX
2021-11-07,Navarra,1046175,1017222,0.972325,522257,NC


In [14]:
#getting data
try :
    final.to_sql(con=engine(), name='vaccins', if_exists='replace')
except : 
    print("Error !")


Error !
